In [1]:
from logicqubit.logic import *
from math import *
import random
import operator
import matplotlib.pyplot as plt
from IPython.display import display, Math, Latex

Cuda is not available!
logicqubit version 1.5.6


In [2]:
def print_matrix(matrix):
    value = sp.latex(sp.Matrix(matrix))
    display(Math(value))

In [139]:
# |<C>| = |<a3b1> + <a1b1> + <a3b3> - <a1b3>| <= 2 for classic systems
def bell_inequality(A, B):
    direction = {"a3b1":0, "a1b1":0, "a3b3":0, "a1b3":0}
    count = {"a3b1":0, "a1b1":0, "a3b3":0, "a1b3":0}
    
    # Expected values for a3b1, a1b1, a3b3 and a1b3
    for a,b in zip(A,B):
        if a[0]=="a3" and b[0]=="b1":
            direction["a3b1"] += a[1]*b[1]
            count["a3b1"] += 1
        elif a[0]=="a1" and b[0]=="b1":
            direction["a1b1"] += a[1]*b[1]
            count["a1b1"] += 1
        elif a[0]=="a3" and b[0]=="b3":
            direction["a3b3"] += a[1]*b[1]
            count["a3b3"] += 1
        elif a[0]=="a1" and b[0]=="b3":
            direction["a1b3"] += a[1]*b[1]
            count["a1b3"] += 1

    direction["a3b1"] /= count["a3b1"]
    direction["a1b1"] /= count["a1b1"]
    direction["a3b3"] /= count["a3b3"]
    direction["a1b3"] /= count["a1b3"]
    # Bell inequality value
    inequality = direction["a3b1"] + direction["a1b1"] + direction["a3b3"] - direction["a1b3"]
    inequality = sqrt(inequality * inequality)
    print(direction)
    return inequality

def measurements(eva=False, attempts=10):
    to_minus_one = lambda x: -1 if x == 0 else x
    measure_A = []
    measure_B = []
  
    for i in range(attempts):
        logicQuBit = LogicQuBit(2)

        A = Qubit()
        B = Qubit()

        # generate entangled state |01>-|10>
        A.H()
        B.CNOT(A)
        B.X()
        B.Z()

        # Alice choose the measurement
        measure_type_A = random.choice(["a1", "a2", "a3"])
        if "a1" == measure_type_A: # for x
            A.H()
        elif "a3" == measure_type_A:
            A.RY(pi/4)
            
        # Alice performs the measurement
        m_A = logicQuBit.Measure_One(A)[0]
        m_A = to_minus_one(m_A)

        # Eva performs the measurement
        if eva:
            measure_type_E = random.choice(["b1", "b2", "b3"])
            if "b1" == measure_type_E:
                B.RY(-pi/4)
            elif "b3" == measure_type_E:
                B.RY(pi/4)
            m_E = logicQuBit.Measure_One(B)[0]

        # Bob choose the measurement
        measure_type_B = random.choice(["b1", "b2", "b3"])
        if "b1" == measure_type_B:
            B.RY(-pi/4)
        elif "b3" == measure_type_B:
            B.RY(pi/4)
       
        # Bob performs the measurement
        m_B = logicQuBit.Measure_One(B)[0]
        m_B = to_minus_one(m_B)

        measure_A.append([measure_type_A, m_A])
        measure_B.append([measure_type_B, m_B])

    return measure_A, measure_B

measure_A, measure_B = measurements(eva=False, attempts=500)
bell = bell_inequality(measure_A, measure_B)
print(bell)

{'a3b1': 0.03333333333333333, 'a1b1': -0.8333333333333334, 'a3b3': -1.0, 'a1b3': 0.6981132075471698}
2.4981132075471697


In [140]:
def perform_measurements(eva = False, attempts=200):
    to_plus_one = lambda x: 0 if x == -1 else x

    measure_A, measure_B = measurements(eva=eva, attempts=attempts)

    # incompatible measures are shared and used to calculate bell inequality
    shared_measures_A = []
    shared_measures_B = []

    # compatible measures will not be shared
    unshared_measures_A = [] 
    unshared_measures_B = []

    # shared measures and types of measures are public information
    for A,B in zip(measure_A,measure_B):
        if (A[0] == 'a3' and B[0] == 'b2') or (A[0] == 'a2' and B[0] == 'b1'):
            unshared_measures_A.append(A)
            unshared_measures_B.append(B)
        else:
            shared_measures_A.append(A)
            shared_measures_B.append(B)

    bell = bell_inequality(shared_measures_A, shared_measures_B)
    print(bell)
    if bell > 2:
        key_A = [to_plus_one(measure[1]) for measure in unshared_measures_A]
        key_B = [int(not to_plus_one(measure[1])) for measure in unshared_measures_B]
        print(key_A)
        print(key_B)
    else:
        print("untrusted key!")

In [141]:
# without eva
perform_measurements(eva = False)

{'a3b1': -0.42857142857142855, 'a1b1': -0.4666666666666667, 'a3b3': -1.0, 'a1b3': 0.6153846153846154}
2.5106227106227106
[0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
[0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0]


In [142]:
# with eva
perform_measurements(eva = True)

{'a3b1': -0.42857142857142855, 'a1b1': 0.0, 'a3b3': -0.6, 'a1b3': -0.14285714285714285}
0.8857142857142857
untrusted key!
